# Introduction

## Teguh Samudra

This project is based on a Kaggle Competition [Titanic Machine Learning from Disaster](https://www.kaggle.com/c/titanic)

# Import Packages

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

In [2]:
raw_data = pd.read_csv('datasets/train.csv')

In [3]:
raw_data.shape

(891, 12)

In [4]:
raw_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


* Cabin column has only about 1/4 of the rows that is non-null value. This column would not be usefull 

* Name is also not a usefull to predict survival.

* Ticket number is not a useful indicator. Instead, we will use Pclass to determine the passenger's class

* Age has some missing values and we will remove these rows from the analysis. 


In [15]:
titanic = raw_data.drop(['Name', 'Cabin', 'Ticket'], axis = 1).dropna(axis = 0, how = 'any')
titanic.shape

(712, 9)

Convert table 'Sex' from Category to binary 0 | 1

In [16]:
ohe = OneHotEncoder(drop = 'first', sparse = False, dtype = int)

ohe_sex = pd.DataFrame(ohe.fit_transform(titanic[['Sex']]), 
                       columns = ohe.categories_[0][1:])
ohe_sex.head()

,male
0,1
1,0
2,0
3,0
4,1


In [17]:
df = pd.concat((titanic, ohe_sex), axis = 1).drop(['Sex'], axis = 1)
df.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,male
0,1.0,0.0,3.0,22.0,1.0,0.0,7.2500,S,1.0
1,2.0,1.0,1.0,38.0,1.0,0.0,71.2833,C,0.0
2,3.0,1.0,3.0,26.0,0.0,0.0,7.9250,S,0.0
3,4.0,1.0,1.0,35.0,1.0,0.0,53.1000,S,0.0
4,5.0,0.0,3.0,35.0,0.0,0.0,8.0500,S,1.0


In [18]:
df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,male
count,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000,712.000000
mean,448.589888,0.404494,2.240169,29.642093,0.514045,0.432584,34.567251,0.636236
std,258.683191,0.491139,0.836854,14.492933,0.930692,0.854181,52.938648,0.481420
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,222.750000,0.000000,1.000000,20.000000,0.000000,0.000000,8.050000,0.000000
50%,445.000000,0.000000,2.000000,28.000000,0.000000,0.000000,15.645850,1.000000
75%,677.250000,1.000000,3.000000,38.000000,1.000000,1.000000,33.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,5.000000,6.000000,512.329200,1.000000


In [25]:
df.shape

(860, 9)

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 860 entries, 0 to 890
Data columns (total 9 columns):
PassengerId    712 non-null float64
Survived       712 non-null float64
Pclass         712 non-null float64
Age            712 non-null float64
SibSp          712 non-null float64
Parch          712 non-null float64
Fare           712 non-null float64
Embarked       712 non-null object
male           712 non-null float64
dtypes: float64(8), object(1)
memory usage: 67.2+ KB


In [30]:
df[df['Parch'] == 'nan']

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked,male


In [24]:
df.Parch.unique()

array([ 0., nan,  1.,  2.,  5.,  3.,  4.,  6.])